In [ ]:
from collections import defaultdict
import unicodedata as ud
import pandas as pd
import numpy as np
import math
import ast
import re

#Dữ liệu thu nhập

In [ ]:
sentences = open('input.txt', encoding='utf-8').readlines()
tokenize_sentences = [sentence.split(' ') for sentence in sentences]

In [ ]:
print('Số lượng câu đã thu thập:', len(sentences))
sentences[0:2]

Số lượng câu đã thu thập: 50


['Đồ ăn hải sản tươi ngon, các món canh nấu rất tuyệt.\n',
 'Mình hay ghé nhà hàng Ngon ở Trần Hưng Đạo để dùng bữa với bạn bè và đối tác.\n']

In [ ]:
max_str = max(tokenize_sentences, key=len)
print('Câu có số từ nhiều nhât:', len(max_str))
' '.join(max_str)

Câu có số từ nhiều nhât: 28


'Vị trí nhà khách thuận lợi rất gần biển nên buổi sáng sớm bạn có thể đi bộ ra biển tập thể dục và tắm biển thoải mái.\n'

In [ ]:
min_str = min(tokenize_sentences, key=len)
print('Câu có số từ ít nhât:', len(min_str))
' '.join(min_str)

Câu có số từ ít nhât: 8


'Giao xe nhanh chóng, thủ tục đơn giản.\n'

#Tách từ

##Longest Matching

In [ ]:
def syllablize(sentence):
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]
def load_n_grams(path):
    with open(path, encoding='utf8') as f:
        words = f.read()
        words = ast.literal_eval(words)
    return words

In [ ]:
def load_n_grams(path):
    with open(path, encoding='utf8') as f:
        words = f.read()
        words = ast.literal_eval(words)
    return words

In [ ]:
def longest_matching(sentence, bi_grams, tri_grams):
    syllables = syllablize(sentence)
    syl_len = len(syllables)
    
    curr_id = 0
    word_list = []
    done = False
    
    while (curr_id < syl_len) and (not done):
        curr_word = syllables[curr_id]
        if curr_id >= syl_len - 1:
            word_list.append(curr_word)
            done = True
        else:
            next_word = syllables[curr_id + 1]
            pair_word = ' '.join([curr_word.lower(), next_word.lower()])
            if curr_id >= (syl_len - 2):
                if pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
            else:
                next_next_word = syllables[curr_id + 2]
                triple_word = ' '.join([pair_word, next_next_word.lower()])
                if triple_word in tri_grams:
                    word_list.append('_'.join([curr_word, next_word, next_next_word]))
                    curr_id += 3
                elif pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
    return word_list

In [ ]:
bi_grams = load_n_grams('vocab/bi_grams.txt')
tri_grams = load_n_grams('vocab/tri_grams.txt')
longest_matching('Mình hay ghé nhà hàng Ngon ở Trần Hưng Đạo để dùng bữa với bạn bè và đối tác.', bi_grams, tri_grams)

['Mình',
 'hay',
 'ghé',
 'nhà_hàng',
 'Ngon',
 'ở_Trần',
 'Hưng',
 'Đạo',
 'để',
 'dùng_bữa',
 'với',
 'bạn_bè',
 'và',
 'đối_tác',
 '.']

In [ ]:
with open('token/Longest_Matching.txt', 'w', encoding='utf-8') as f:
    longest_matching_sentences = []
    for sentence in sentences:
        word_list = longest_matching(sentence, bi_grams, tri_grams)
        longest_matching_sentences.append(' '.join(word_list))
        for word in word_list: f.write(word + '\n')
        if sentence != sentences[-1]: f.write('\n')
    f.write('\n')
longest_matching_sentences[0:3]

['Đồ ăn hải_sản tươi ngon , các món canh nấu rất tuyệt .',
 'Mình hay ghé nhà_hàng Ngon ở_Trần Hưng Đạo để dùng_bữa với bạn_bè và đối_tác .',
 'Đồ ăn tươi ngon chế_biến vừa_vặn , giá_cả hợp_lý cạnh quán Nghĩa Ghẹ .']

In [ ]:
count_longest_matching_compounds = 0
for sentence in longest_matching_sentences:
    for word in sentence.split():
        if '_' in word: count_longest_matching_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching:', count_longest_matching_compounds)

Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching: 197


##VnCoreNLP

In [ ]:
!pip install py_vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 4.9 MB/s 
  Created wheel for py-vncorenlp: filename=py_vncorenlp-0.1.3-py3-none-any.whl size=4307 sha256=9fc605b9af5d43cc922093afd132dd9ff67efaac6f1283ee807d0f4306e499c0
  Stored in directory: /root/.cache/pip/wheels/d7/73/13/6b1f2c14831151fc1417f3dc1fa6b241e27ad13f8c83691753
Successfully built py-vncorenlp


In [ ]:
import py_vncorenlp

In [ ]:
py_vncorenlp.download_model(save_dir='./')

In [ ]:
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='./')

In [ ]:
text = "Mình hay ghé nhà hàng Ngon ở Trần Hưng Đạo để dùng bữa với bạn bè và đối tác."
output = rdrsegmenter.word_segment(text)
print(output)

['Mình hay ghé nhà_hàng Ngon ở Trần_Hưng_Đạo để dùng_bữa với bạn_bè và đối_tác .']


In [ ]:
with open('token/vncore.txt', 'w', encoding='utf-8') as f:
    vncore_sentences = []
    for sentence in sentences:
        word_list = rdrsegmenter.word_segment(sentence)
        vncore_sentences.append(' '.join(word_list))
        for word in word_list: f.write(word)
        if sentence != sentences[-1]: f.write('\n')
    f.write('\n')
vncore_sentences[0:3]

['Đồ_ăn hải_sản tươi ngon , các món canh nấu rất tuyệt .',
 'Mình hay ghé nhà_hàng Ngon ở Trần_Hưng_Đạo để dùng_bữa với bạn_bè và đối_tác .',
 'Đồ_ăn tươi ngon chế_biến vừa_vặn , giá_cả hợp_lý cạnh quán Nghĩa Ghẹ .']

In [ ]:
count_vncore_compounds = 0
for sentence in vncore_sentences:
    for word in sentence.split():
        if '_' in word: count_vncore_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP:', count_vncore_compounds)

Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP: 211


##Thủ công

In [ ]:
with open('token/vietnamese.txt', 'r', encoding='utf-8') as f:
    manual_tokenize_sentences = []
    sentence = ''
    for word in f:
        if word == '\n': 
            manual_tokenize_sentences.append(sentence.strip())
            sentence = ''
        else: sentence += word.replace('\n', ' ')
manual_tokenize_sentences[0:3]

['Đồ_ăn hải_sản tươi ngon , các món canh nấu rất tuyệt .',
 'Mình hay ghé nhà_hàng Ngon ở Trần_Hưng_Đạo để dùng bữa với bạn_bè và đối_tác .',
 'Đồ_ăn tươi ngon chế_biến vừa_vặn , giá_cả hợp_lý cạnh quán Nghĩa_Ghẹ .']

In [ ]:
count_manual_tokenize_compounds = 0
for sentence in manual_tokenize_sentences:
    for word in sentence.split():
        if '_' in word: count_manual_tokenize_compounds += 1
print('Số lượng từ ghép khi tách từ thủ công:', count_manual_tokenize_compounds)

Số lượng từ ghép khi tách từ thủ công: 206


#Đánh giá kết quả tách từ


In [ ]:
def count_correct_words(pred, source, n_grams=3):
    pred_words = pred.split()
    source_words = source.split()
    
    total_true, tp = 0, 0
    total_errors, fp = 0, 0
    
    idx = 0
    while idx < len(pred_words):
        if pred_words[idx] not in source_words[idx:(idx + n_grams)]: 
            if '_' in pred_words[idx]: fp += 1
            del pred_words[idx]
            total_errors += 1
        else: idx += 1
    
    idx = 0
    while idx < len(source_words):
        if source_words[idx] not in pred_words[idx:(idx + n_grams)]: 
            del source_words[idx]
        else: idx += 1
    
    if len(pred_words) < len(source_words): words = pred_words
    else: words = source_words
    
    for idx in range (len(words)):
        if pred_words[idx] == source_words[idx]:
            if '_' in pred_words[idx]: tp += 1 
            total_true += 1
                    
    return total_true, total_errors, tp, fp

In [ ]:
def tokenize_evaluation(pred, source, n_grams=3):
    total_true = 0
    total_errors = 0
    total_words = 0
    
    pred_tp = 0
    pred_fp = 0
    
    for pred_sentence, source_sentence in zip(pred, source):
        total_words += len(source_sentence.split())
        if pred_sentence != source_sentence:
            true, error, tp, fp = count_correct_words(pred_sentence, source_sentence, n_grams)
            total_true += true 
            total_errors += error
            pred_tp += tp
            pred_fp += fp
        else:
            for word in source_sentence.split():
                if '_' in word: pred_tp += 1
                total_true += 1
    return {
        'Accuracy': total_true / total_words, 
        'Precision': pred_tp / (pred_tp + pred_fp),
        'Recall': pred_tp / count_manual_tokenize_compounds,
        'True Positive': pred_tp, 
        'False Positive': pred_fp,
        'Total True': total_true, 
        'Total Errors': total_errors
    }

In [ ]:
longest_matching_evaluation = tokenize_evaluation(longest_matching_sentences, manual_tokenize_sentences)
vncore_evaluation = tokenize_evaluation(vncore_sentences, manual_tokenize_sentences)
pd.DataFrame(
    [longest_matching_evaluation, vncore_evaluation], 
    index = ['Longest Matching', 'VnCoreNLP']
).astype(object).T

,Longest Matching,VnCoreNLP
Accuracy,0.930362,0.945682
Precision,0.938144,0.947115
Recall,0.883495,0.956311
True Positive,182,197
False Positive,12,11
Total True,668,679
Total Errors,66,40


In [ ]:
longest_matching_tokens = open('token/Longest_Matching.txt', encoding='utf-8').readlines()
print('Số lượng từ:', len(longest_matching_tokens))
longest_matching_tokens[0:5]

Số lượng từ: 800


['Đồ\n', 'ăn\n', 'hải_sản\n', 'tươi\n', 'ngon\n']

In [ ]:
vncore_tokens = open('token/vncore.txt', encoding='utf-8').readlines()
print('Số lượng từ:', len(vncore_tokens))
vncore_tokens[0:5]

Số lượng từ: 783


['Đồ_ăn\n', 'hải_sản\n', 'tươi\n', 'ngon\n', ',\n']

In [ ]:
manual_tokens = open('token/vietnamese.txt', encoding='utf-8').readlines()
print('Số lượng từ:', len(manual_tokens))
manual_tokens[0:5]

Số lượng từ: 783


['Đồ_ăn\n', 'hải_sản\n', 'tươi\n', 'ngon\n', ',\n']

In [ ]:
with open('dataset/postag.txt', 'w', encoding='utf-8') as f:
    for word in manual_tokens:
        word = word.replace('\n', '')
        
        if '_' not in word: tag = rdrsegmenter.pos_tag(word)
        else: tag = rdrsegmenter.pos_tag(word.replace('_', ' '))
        
        if tag == []: f.write('\n')
        else: f.write(f'{word}\t{tag[0][0][1]}\n')
    f.write('\n')

AttributeError: ignored